In [24]:
import json
import uuid
import random
import hashlib
from pathlib import Path

# 路径配置（符合你的项目结构）
DATA_DIR = Path("data")
USERS_PATH = DATA_DIR / "users.json"

# 偏好环境可选值
ENV_OPTIONS = ["mountain", "lake", "beach", "city"]

# 工具函数：生成 SHA256 哈希
def sha256(text: str) -> str:
    return hashlib.sha256(text.encode("utf-8")).hexdigest()

# 随机生成姓名
def random_name():
    first_names = [
        "Alex","Sam","Chris","Taylor","Jordan","Jamie","Casey","Robin","Drew","Jesse",
        "Lee","Avery","Quinn","Riley","Charlie","Morgan","Skyler","Parker","Rowan","Shawn"
    ]
    last_names = [
        "Chen","Lee","Wang","Patel","Smith","Jones","Brown","Garcia","Martin","Nguyen",
        "Kim","Singh","Lam","Hernandez","Wilson","Anderson","Thomas","Moore","Clark","Young"
    ]
    return f"{random.choice(first_names)} {random.choice(last_names)}"

# 生成 1000 条用户记录
def generate_users(n=1000):
    DATA_DIR.mkdir(parents=True, exist_ok=True)
    users = []
    for i in range(n):
        username = f"user{i:04d}"  # user0000 ~ user0999
        password_hash = sha256(username)  # 为演示方便，密码设为同用户名
        budget_min = random.choice([80, 100, 120, 150, 180])
        budget_max = budget_min + random.choice([50, 80, 100, 150])

        users.append({
            "user_id": f"u_{uuid.uuid4().hex[:8]}",  # 唯一 ID
            "username": username,
            "password_hash": password_hash,
            "name": random_name(),
            "group_size": random.randint(1, 6),
            "preferred_env": random.choice(ENV_OPTIONS),
            "budget_min": budget_min,
            "budget_max": budget_max,
            "travel_dates": None  # 可选字段
        })

    # 保存到 JSON 文件（覆盖原文件）
    with open(USERS_PATH, "w", encoding="utf-8") as f:
        json.dump(users, f, indent=2, ensure_ascii=False)
    print(f"✅ Generated {n} users → {USERS_PATH}")

if __name__ == "__main__":
    generate_users()


✅ Generated 1000 users → data\users.json


In [15]:
## Step 1: Read Properties
# 在文件顶部的 import 后、工具函数区附近加入：
from typing import List, Dict

PROPS_PATH = r"C:\Users\vicky\OneDrive - University of Toronto\MMA\property_data.json"


def read_properties() -> List[Dict]:
    if not PROPS_PATH.exists():
        return []
    with open(PROPS_PATH, "r", encoding="utf-8") as f:
        return json.load(f)

def filter_properties(props: List[Dict],
                      env: str = "",
                      price_min: int = None,
                      price_max: int = None,
                      capacity: int = None,
                      tags_or_features: List[str] = None) -> List[Dict]:
    """按环境/价格/容量/标签特征进行筛选（都为可选条件）。"""
    tags_or_features = [t.strip().lower() for t in (tags_or_features or []) if t.strip()]
    out = []
    for p in props:
        if env and p.get("environment","").lower() != env.lower():
            continue
        price = int(p.get("nightly_price", 0))
        if price_min is not None and price < price_min:
            continue
        if price_max is not None and price > price_max:
            continue
        if capacity is not None and int(p.get("capacity", 0)) < capacity:
            continue
        # 标签/特征为“或”匹配：任一命中即可（你也可以改成“与”）
        if tags_or_features:
            have = set([x.lower() for x in p.get("features", [])] + [x.lower() for x in p.get("tags", [])])
            if have.isdisjoint(tags_or_features):
                continue
        out.append(p)
    return out


In [7]:
## Steo 2: Proceed to menu, user could choose to filter on Properties
def filter_menu(user: dict):
    """
    登录后的房源筛选界面。所有条件都可跳过（按回车）。
    """
    props = read_properties()
    if not props:
        print("⚠️ 没有房源数据（data/properties.json 不存在或为空）。")
        return

    print("\n=== Filter Listings ===")
    env = input("Environment [mountain/lake/beach/city] (Enter=skip): ").strip().lower()
    price_min = input("Min price/night (Enter=skip): ").strip()
    price_max = input("Max price/night (Enter=skip): ").strip()
    capacity  = input(f"Min capacity (default {user['group_size']}, Enter=use default): ").strip()
    tags = input("Tags/Features (comma-separated, e.g., wifi,hot_tub,remote) (Enter=skip): ").strip()

    price_min = int(price_min) if price_min else None
    price_max = int(price_max) if price_max else None
    capacity  = int(capacity) if capacity else user["group_size"]
    tags_list = [t for t in tags.split(",")] if tags else []

    results = filter_properties(props, env=env, price_min=price_min,
                                price_max=price_max, capacity=capacity,
                                tags_or_features=tags_list)

    if not results:
        print("😶 没有符合条件的结果。")
        return

    print(f"\n✅ 找到 {len(results)} 条：")
    for p in results[:20]:   # 显示前20条
        print(f"- {p.get('property_id','?')} | {p.get('location','?')} | "
              f"{p.get('environment','?')} | ${p.get('nightly_price','?')}/night | "
              f"cap={p.get('capacity','?')} | features={p.get('features',[])} | tags={p.get('tags',[])}")
    if len(results) > 20:
        print(f"... 还有 {len(results)-20} 条未显示")


In [9]:
## Step 3: After searching complete, exit to main menu where user can both view/edit/delete profile and once again filter properties
def after_login(user: dict):
    """
    登录后总菜单：既能筛选房源，也能管理个人资料。
    """
    while True:
        print("\n=== Logged in as:", user["username"], "===")
        print("Actions: [filter]  [profile]  [logout]")
        act = input("> ").strip().lower()

        if act == "filter":
            filter_menu(user)

        elif act == "profile":
            status = profile_menu(user)
            if status == "deleted":
                # 用户被删除，回主菜单
                break

        elif act == "logout":
            print("Logged out.")
            break

        else:
            print("Unknown action.")


In [ ]:
import json
import uuid
import random
import hashlib
from pathlib import Path

# 输出文件路径
DATA_DIR = Path("data")
USERS_PATH = DATA_DIR / "users.json"

# 偏好环境可选值
ENV_OPTIONS = ["mountain", "lake", "beach", "city"]

# 工具函数：SHA256 密码哈希
def sha256(text: str) -> str:
    return hashlib.sha256(text.encode("utf-8")).hexdigest()

# 随机姓名生成
def random_name():
    first = random.choice(
        ["Alex","Sam","Chris","Taylor","Jordan","Jamie","Casey","Robin","Drew","Jesse",
         "Lee","Avery","Quinn","Riley","Charlie","Morgan","Skyler","Parker","Rowan","Shawn"]
    )
    last = random.choice(
        ["Chen","Lee","Wang","Patel","Smith","Jones","Brown","Garcia","Martin","Nguyen",
         "Kim","Singh","Lam","Hernandez","Wilson","Anderson","Thomas","Moore","Clark","Young"]
    )
    return f"{first} {last}"

# 生成用户数据
def generate_users(n=1000):
    DATA_DIR.mkdir(parents=True, exist_ok=True)
    users = []
    for i in range(n):
        uname = f"user{i:04d}"
        pwd_hash = sha256(uname)  # 为演示方便，密码设成和用户名一样
        budget_min = random.choice([80, 100, 120, 150, 180])
        budget_max = budget_min + random.choice([50, 80, 100, 150])
        users.append({
            "user_id": f"u_{uuid.uuid4().hex[:8]}",
            "username": uname,
            "password_hash": pwd_hash,
            "name": random_name(),
            "group_size": random.randint(1, 6),
            "preferred_env": random.choice(ENV_OPTIONS),
            "budget_min": budget_min,
            "budget_max": budget_max,
            "travel_dates": None
        })
    with open(USERS_PATH, "w", encoding="utf-8") as f:
        json.dump(users, f, indent=2, ensure_ascii=False)
    print(f"✅ Generated {n} users → {USERS_PATH}")

if __name__ == "__main__":
    generate_users()


✅ Generated 1000 users → data\users.json


In [13]:
import os
print(f"File saved at: {os.path.abspath('users.json')}")

File saved at: c:\Users\vicky\OneDrive - University of Toronto\MMA\users.json
